# Install the required libraries

This cell installs the required libraries, 'xmltodict' and 'pyyaml', if you haven't installed them already. Running this cell is optional if you already have the libraries installed.

In [ ]:
!pip install xmltodict pyyaml

## Import libraries and define the `update_item` function

This cell imports the necessary libraries and defines the 'update_item' function that updates item properties such as factor, restock, lifetime, and tier based on the provided configuration.

In [ ]:
import xml.etree.ElementTree as ET
import yaml
import io

def update_item(root, item_config):
    item_name = item_config['name']
    factor = item_config.get('factor', 1)
    restock = item_config.get('restock')
    lifetime = item_config.get('lifetime')
    tier_changes = item_config.get('tier', [])

    item_found = False

    for item in root.iter('type'):
        if item.get('name') == item_name:
            item_found = True

            # Update nominal value
            nominal_element = item.find('nominal')
            if nominal_element is not None:
                nominal = int(nominal_element.text)
                new_nominal = int(nominal * factor)
                nominal_element.text = str(new_nominal)
            else:
                print(f"Warning: Item '{item_name}' does not have a nominal value")


            # Update restock and lifetime values if provided
            if restock is not None:
                item.set('restock', str(restock))
            if lifetime is not None:
                item.set('lifetime', str(lifetime))

            # Update tier values
            if not isinstance(tier_changes, list):
                tier_changes = [tier_changes]
            for i in range(1, 5):
                if i in tier_changes:
                    item.set(f'tier{i}', '1')
                else:
                    item.set(f'tier{i}', '0')

    if not item_found:
        print(f"Warning: Item '{item_name}' not found in types.xml")

## Read the original `types.xml` file:
This cell reads the original 'types.xml' file, which contains the item configuration for the game. Make sure to place the 'types.xml' file in the same directory as the notebook.

In [ ]:
with open('types.xml', 'r') as file:
 tree = ET.parse(file)
 root = tree.getroot()

## Install the ipywidgets library

This cell installs the 'ipywidgets' library, which is required to create interactive widgets in the notebook. Running this cell is optional if you already have the library installed.

In [ ]:
!pip install ipywidgets

## Import libraries and create a function to load the YAML configuration

This cell imports the necessary libraries and defines the 'load_config' function that reads the YAML configuration file and returns the configuration as a dictionary.

In [ ]:
import yaml

def load_config(config_filename):
    with open(config_filename, 'r') as file:
        data = yaml.safe_load(file)
    return data['items']

## List available YAML configuration files and create a dropdown widget

This cell lists all available YAML configuration files in the same directory as the notebook and creates a dropdown widget that allows you to select a configuration file.

In [ ]:
config_files = [file for file in os.listdir() if file.endswith('.yaml')]

dropdown = widgets.Dropdown(
 options=config_files,
 description='Config File:',
 disabled=False,
)

display(dropdown)

## Load the selected configuration file or the default one

This cell checks if a configuration file has been selected using the dropdown widget. If a file has been selected, it loads the configuration from that file. If not, it loads a default configuration file. Make sure to replace 'default_config.yaml' with the actual name of your default configuration file.

In [ ]:
if dropdown.value is None:
    config_filename = 'default_config.yaml'  # Replace this with your default config filename
    print(f"Loading default configuration file: {config_filename}")
else:
    config_filename = dropdown.value
    print(f"Loading selected configuration file: {config_filename}")

config = load_config(config_filename)

## Update the nominal values for the items in the config

This cell updates the nominal values for the items in the selected configuration using the 'update_item' function.

In [ ]:
for item_config in config:
 update_item(root, item_config)

## Save the updated XML to a new file

This cell saves the updated XML to a new file named 'updated_types.xml'. You can use this file to modify the game's item spawning behavior according to the selected configuration.

In [ ]:
with open('updated_types.xml', 'wb') as file:
 tree.write(file)